In [0]:
# What's new in v0.9?

# 1) Calculation of stat.tests for medians difference for continuous metrics
# 2) Calculation of ZPS KPIs by default (GMVbR, Num Orders, Num Purchase attempts,PACR,PSSR,PPSR)

In [0]:
%run /PayDayZPSTeam/_lib/scala/helpers


In [0]:
 %run /PayDayZPSTeam/_lib/python/helpers

In [0]:
%run /../ab_tests_related_functions_v2

Input configurations for the experiment calculations

In [0]:
# inputs:

experiment_name = 'Test_of_new_feature_for_notebook_repo_fin' # Name of experiment. Try to make it unique
test_start_date = '2022-06-10' # First date of roll-out of the experiment
calc_start_date = '2022-06-12' # First date of results calculation. Results well be calculate from test_start_date up to date between calc_start_date and calc_end_date.
calc_end_date = '2022-06-20' #If you need to calculate results just for 1 day make calc_start_date=calc_end_date
# yesterday=(pd.to_datetime("today") - pd.Timedelta(1, unit='D')).strftime('%Y-%m-%d')
calc_frequency = '7d' # frequency of calculations between calc_start_date and calc_end_date. Example: if calc_frequency='7d' and calc_start_date ='2022-01-01' and calc_end_date='2022-01-22' then calculations will be performed on: 2022-01-01, 2022-01-08,2022-01-15,2022-01-22. More info on frequency types: https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases 
dates=['2022-07-20', '2022-07-22'] # you can input list of dates to calculate result for each iteratively e.g. for [test_start_date;date_1],[test_start_date,date_2].. if yo want to calculate results daily or by given frequency just leave it empty. Note that number of dates in dates list should be > 2

metrics_columns = ['gmvbr','num_purchase_attempts','num_orders',{'gmvbr':'median'}] # Names of metrics you need to calculate stat.tests. Should be the same as names in sql query results table.

proportions_metrics_flag = True # if you need to calculate proportion metrics in 'numerator/denumerator' format
proportions_columns = {'num_customers_with_morethan2_orders':'all_customers'} # names of metrics from sql query that you want to use for proportion metrics in numerator:denumerator format. Note that values per user must be binary 0 or 1. 

ratio_metrics_flag = True # if you need to calculate ratio metrics in 'numerator/denumerator' format
ratio_columns = {'num_orders':'num_purchase_attempts'} # names of metrics from sql query that you want to use for ratio metrics in numerator:denumerator format. 

need_bootstrap = True # True if you want to calculate Bootstrap statistics 
need_boostrap_wo_outliers = True # True if you want to calculate Bootstrap statistics based on data with removed outliers
need_buckets = False # True if you want to calculate ttest based on buckets split
need_buckets_wo_outliers = False # True if you want to calculate ttest based on buckets split of data with removed outliers
is_zps_kpis_list_needed = True # True if you want to get ZPS KPIs (GMV,PACR,PSSR,PPSR) calculated by default

quantile_for_outliers = 0.99999 # quantile for outliers removing. Users who have metric value above this quentile will be removed for 'without outliers' calculations
bootstrap_iterations = 1000 # number of iterations in bootstrap. You can 
buckets_num = 150 # number of buckets to split (more buckets -> less customers in one bucket -> less precision; less buckets -> less sample for ttest -> less precision, so it is always a tradeoff. Recomendation is not to use less than 60 buckets and have number of customers in bucket not less than 4000)

customer_id_column = 'customer_id' #name of column in sql resulted table that contains customer IDs
variant_col = 'payload_variant' #name of column in sql resulted table that contains names of treatment and control groups
control_gr_name = 'Invoice' #name of control group in variant_col
treatment_gr_name = 'BNPL2.0' #name of treatment group in variant_col

In [0]:
%scala
// Input name of table where rusults should be placed (table should be created before the start of calculations. if not - uncomment and run cmd 7)
val TargetTable = "zps_analytics.ab_test_calculations_new_approach"

In [0]:
%scala
// create table for stat.tests results collection on RedShift if not yet created:

// val TargetTable = "zps_analytics.ab_test_calculations_new_approach"

// dwhSend(s"""
//   -- Want to rebuild the full table? Then uncomment the next line. :)
//   drop table if exists $TargetTable;
//   create table if not exists $TargetTable (
//        experiment_name varchar(255)
//        ,dt date
//        ,metric varchar(255)
//        ,metric_type varchar(255)
//        ,mean_control float
//        ,mean_treatment float
//        ,mean_diff float
//        ,mean_control_wo_outliers float
//        ,mean_treatment_wo_outliers float
//        ,mean_diff_wo_outliers float
//        ,median_control float
//        ,median_treatment float
//        ,median_diff float
//        ,ttest_pval float
//        ,ttest_mean_diff float
//        ,ttest_ci_prc_low float
//        ,ttest_ci_prc_up float
//        ,ttest_wo_outliers_pval float
//        ,ttest_wo_outliers_mean_diff float
//        ,ttest_wo_outliers_ci_prc_low float
//        ,ttest_wo_outliers_ci_prc_up float
//        ,btstrp_pval float
//        ,btstrp_mean_diff float
//        ,btstrp_ci_prc_low float
//        ,btstrp_ci_prc_up float
//        ,btstrp_wo_outliers_pval float
//        ,btstrp_wo_outliers_mean_diff float
//        ,btstrp_wo_outliers_ci_prc_low float
//        ,btstrp_wo_outliers_ci_prc_up float
//        ,buckets_pval float
//        ,buckets_mean_diff float
//        ,buckets_ci_prc_low float
//        ,buckets_ci_prc_up float
//        ,buckets_wo_outliers_pval float
//        ,buckets_wo_outliers_mean_diff float
//        ,buckets_wo_outliers_ci_prc_low float
//        ,buckets_wo_outliers_ci_prc_up float
//        ,MW_pval float
//        ,chi2_pval float                        
//        ,outliers_percentile float
//        ,outliers_threshold float
//        ,calculations_time varchar(255)
//        ,mde_bootstrap float
//        ,mde_ttest_wo_outliers float
//        ,units_control float
//        ,units_treatment float
//        ,srm_pval float
//   );
  
//   -- share with all Redshift users (mark as comment if not wished)
//   grant select on $TargetTable to public
// """)

//  // if you need to add new column:
// val sqlAdd = s"ALTER TABLE $TargetTable ADD COLUMN srm_pval float DEFAULT Null"
// dwhSend(sqlAdd)

// if you need to remove rows:
// val sqlDelete = s"DELETE FROM $TargetTable WHERE experiment_name = 'Test_of_new_feature_for_notebook_2'  "
// dwhSend(sqlDelete)

Insert SQL query in cmd9 that counts needed metrics grouped by customer_id and variant type. 
Use date parameters from config ('test_start_date' and 'day' as date between calc_start_date and calc_end_date):

In [0]:
df_results=pd.DataFrame()
  # create a list of dates for calculations
if len(dates)<3:
  dates=pd.date_range(calc_start_date,calc_end_date,freq=calc_frequency).strftime('%Y-%m-%d').tolist()
calculations_time=pd.to_datetime("today").strftime('%Y-%m-%d-%H-%M-%S')
print(dates)
for day in dates:

#  Actions needed!
#   insert your query with data grouped by customer_id and group type (control/treatment). Add date parameters for test_start_date and day of calculations.
    
    query= f"""
with test_assignment as (
        --group assignment info from octopus
        select
            customer_number as customer_id
            , min(payload_variant) as payload_variant
            , min(occurred_at) as assigned_at
            from ods.ods_f_octopus_feedback
        where payload_experiment_id in (
            'f1e6574c-a5a5-4f51-be73-c472e3bc91b9'
            , 'b9b3b401-da13-434f-b970-224ce3b9c01e'
        )  --experiment_id for BNPL DE
         and payload_mode like 'TEST'
        group by 1

    )
     select ta.customer_id
         , ta.payload_variant
         , coalesce(sum(gmv_bef_cancellation), 0)::float as gmvbr
         , coalesce(count(purchase_attempt_id), 0) as num_purchase_attempts
         , coalesce(sum(num_orders_placed), 0) as num_orders
         , 1 as all_customers
         , case when num_orders>1 then 1 else 0 end as num_customers_with_morethan2_orders
    from reporting.pam_purchase_attempt_funnel cm
    inner join test_assignment as ta using(customer_id)
    where purchase_attempt_created_at::date between '"""+test_start_date+"""' and '"""+day+"""' and sales_channel='Shop DE'
    group by 1,2

"""
    query_flat = None  #you can write here additional query, for more information https://docs.google.com/document/d/1jh4f5k_n13ioJTQFUKj1eX0eEpKiKVXnacj35fmm1oI/edit#heading=h.z71pmqeqe5dw
    df_day_results = main_ab_test_calculation_function(customer_id_column, test_start_date, day, calc_start_date, calc_end_date, metrics_columns, need_bootstrap, need_buckets, proportions_metrics_flag, proportions_columns, ratio_metrics_flag, ratio_columns, query, is_zps_kpis_list_needed, query_flat=query_flat)
    df_results = df_results.append(df_day_results, ignore_index=False)

df_results = df_results.sort_values(by=['metric'])
df_to_redshift = spark.createDataFrame(df_results)
df_to_redshift.createOrReplaceTempView("tmp")
df_results.head()


In [0]:
%scala
// change nothing here, just run it
var df_to_redshift_final = sql("""
select      
   experiment_name::string
   ,cast(dt as date) dt
   ,metric::string
   ,metric_type::string
   ,mean_control::double
   ,mean_treatment::double
   ,mean_diff::double
   ,mean_control_wo_outliers::double
   ,mean_treatment_wo_outliers::double
   ,mean_diff_wo_outliers::double
   ,median_control::double
   ,median_treatment::double
   ,median_diff::double
   ,ttest_pval::double
   ,ttest_mean_diff::double
   ,ttest_ci_prc_low::double
   ,ttest_ci_prc_up::double
   ,ttest_wo_outliers_pval::double
   ,ttest_wo_outliers_mean_diff::double
   ,ttest_wo_outliers_ci_prc_low::double
   ,ttest_wo_outliers_ci_prc_up::double
   ,btstrp_pval::double
   ,btstrp_mean_diff::double
   ,btstrp_ci_prc_low::double
   ,btstrp_ci_prc_up::double
   ,btstrp_wo_outliers_pval::double
   ,btstrp_wo_outliers_mean_diff::double
   ,btstrp_wo_outliers_ci_prc_low::double
   ,btstrp_wo_outliers_ci_prc_up::double
   ,buckets_pval::double
   ,buckets_mean_diff::double
   ,buckets_ci_prc_low::double
   ,buckets_ci_prc_up::double
   ,buckets_wo_outliers_pval::double
   ,buckets_wo_outliers_mean_diff::double
   ,buckets_wo_outliers_ci_prc_low::double
   ,buckets_wo_outliers_ci_prc_up::double
   ,MW_pval::double
   ,chi2_pval::double                        
   ,outliers_percentile::double
   ,outliers_threshold::double
   ,calculations_time::string
   ,mde_bootstrap::double
   ,mde_ttest_wo_outliers::double
   ,units_control::double
   ,units_treatment::double
   ,srm_pval::double
from tmp""")
display(df_to_redshift_final)

In [0]:
get_summary(df_results)

In [0]:
%scala
//  write tests results in Redshift TargetTable defined in cmd 6 
val sqlPreRun = s"select * from $TargetTable"
dwhWrite(df_to_redshift_final, TargetTable, sqlPreRun)